In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import rarfile
import glob
import shutil
import time
import re
import psycopg2
from psycopg2 import extras
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# --- CONFIGURACIÓN DE RUTA ESPECÍFICA ---
CIUDAD_A_FILTRAR = "HERMOSILLO"
ESTADO_A_FILTRAR = "SONORA"

# Definimos la carpeta base donde tú quieres los datos
RUTA_BASE = r"D:\Documentos\ICD"

# Definimos el nombre de la subcarpeta temporal
NOMBRE_CARPETA_TEMP = "datos_profeco_temp"

# Combinamos ambas para crear la ruta completa segura: D:\Documentos\ICD\datos_profeco_temp
CARPETA_DATOS = os.path.join(RUTA_BASE, NOMBRE_CARPETA_TEMP)

# Verificamos si existe la carpeta madre (ICD), si no, avisamos
if not os.path.exists(RUTA_BASE):
    print(f"⚠️ OJO: La carpeta '{RUTA_BASE}' no parece existir. Python intentará crear toda la ruta.")

print(f"✅ Configuración lista.")
print(f"📂 Los archivos se guardarán en: {CARPETA_DATOS}")

✅ Configuración lista.
📂 Los archivos se guardarán en: D:\Documentos\ICD\datos_profeco_temp


In [6]:
print(f"🚀 Iniciando descarga en Disco D (2020-2025)...")
start_time = time.time()

# 1. Obtener enlaces
url_pagina_links = "https://datos.profeco.gob.mx/datos_abiertos/qqp.php"
url_base_descarga = "https://datos.profeco.gob.mx/datos_abiertos/"

try:
    r = requests.get(url_pagina_links)
    soup = BeautifulSoup(r.text, 'html.parser')
    links = soup.select("li > a")
except Exception as e:
    print(f"❌ Error obteniendo enlaces: {e}")
    links = []

# 2. Bucle de Descarga
if not os.path.exists(CARPETA_DATOS):
    os.makedirs(CARPETA_DATOS)

for link in links:
    texto_link = link.text.strip()
    year_str = texto_link[-4:]
    
    if not year_str.isdigit(): continue
    year = int(year_str)

    if year < 2020 or year > 2025: continue

    print(f"\n📂 Procesando Año: {year}")
    
    nombre_rar = f"QQP_{year}.rar"
    ruta_rar = os.path.join(CARPETA_DATOS, nombre_rar)
    carpeta_destino_año = os.path.join(CARPETA_DATOS, f"EXTRACT_{year}")

    # A. Descargar (si no existe la carpeta ya extraída)
    if not os.path.exists(carpeta_destino_año):
        if not os.path.exists(ruta_rar):
            print(f"   ⬇️ Descargando RAR en D:...")
            try:
                contenido = requests.get(url_base_descarga + link["href"]).content
                with open(ruta_rar, 'wb') as f:
                    f.write(contenido)
            except Exception as e:
                print(f"   ❌ Error descarga: {e}")
                continue
        
        # B. Extraer
        print(f"   📦 Extrayendo en D:...")
        try:
            os.makedirs(carpeta_destino_año, exist_ok=True)
            with rarfile.RarFile(ruta_rar, 'r') as f:
                f.extractall(path=carpeta_destino_año)
            
            # TRUCO PARA AHORRAR ESPACIO:
            # Borramos el RAR en cuanto terminamos de descomprimir
            print("   🗑️ Borrando archivo RAR para liberar espacio...")
            os.remove(ruta_rar)
            
        except Exception as e:
            print(f"   ❌ Error extracción: {e}")
    else:
        print(f"   ℹ️ Los datos del {year} ya están extraídos en D:. Saltando descarga.")

print(f"\n✅ Archivos listos en {CARPETA_DATOS}")

🚀 Iniciando descarga en Disco D (2020-2025)...

📂 Procesando Año: 2025
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   🗑️ Borrando archivo RAR para liberar espacio...

📂 Procesando Año: 2025
   ℹ️ Los datos del 2025 ya están extraídos en D:. Saltando descarga.

📂 Procesando Año: 2024
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   🗑️ Borrando archivo RAR para liberar espacio...

📂 Procesando Año: 2023
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   ❌ Error extracción: Failed the read enough data: req=1048576 got=71462

📂 Procesando Año: 2022
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   ❌ Error extracción: Failed the read enough data: req=1048576 got=442806

📂 Procesando Año: 2021
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   ❌ Error extracción: Failed the read enough data: req=1048576 got=316765

📂 Procesando Año: 2020
   ⬇️ Descargando RAR en D:...
   📦 Extrayendo en D:...
   ❌ Error extracción: Failed the read enough data: r

In [10]:
print("🛠️ Configurando Base de Datos con credenciales manuales...")

load_dotenv()

# Configuración de credenciales de base de datos usando variables de entorno
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")

try:
    conn = psycopg2.connect(
        user=DB_USER,
        password=DB_PASS,
        host=DB_HOST,
        port=5432,
        dbname=DB_NAME
    )
    
    conn.autocommit = False 
    cur = conn.cursor()
    print("✅ ¡Conexión exitosa!")

    # 3. Crear Tabla Maestra
    query_create_table = """
    CREATE TABLE IF NOT EXISTS historico_precios_canasta (
        id SERIAL PRIMARY KEY,
        producto_categoria VARCHAR(255),
        presentacion_original VARCHAR(255),
        marca VARCHAR(255),
        categoria_general VARCHAR(255),
        precio_lista FLOAT,
        fecha_registro DATE,
        nombre_tienda VARCHAR(255),
        direccion TEXT,
        estado VARCHAR(100),
        municipio VARCHAR(100),
        latitud FLOAT,
        longitud FLOAT,
        -- Columnas calculadas
        cantidad_estandarizada FLOAT,
        unidad_estandarizada VARCHAR(10),
        precio_unitario FLOAT
    );
    """
    cur.execute(query_create_table)
    conn.commit()
    print("✅ Tabla verificada correctamente.")

except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("👉 PISTA: ¿Seguro que escribiste bien tu contraseña en la variable DB_PASS?")

🛠️ Configurando Base de Datos con credenciales manuales...
✅ ¡Conexión exitosa!
✅ Tabla verificada correctamente.


In [12]:
print("🚀 Iniciando procesamiento inteligente y carga a SQL...")

# --- A. Funciones de Limpieza (Regex) ---
def extraer_cantidad(texto):
    # Busca números (ej: 900, 1.5, 0.250)
    match = re.search(r'(\d+\.?\d*)', str(texto))
    return float(match.group(1)) if match else 1.0

def extraer_unidad(texto):
    texto = str(texto).lower()
    # Busca patrones de unidades comunes
    if re.search(r'kg|kilo', texto): return 'kg'
    if re.search(r'g|gr', texto): return 'gr'
    if re.search(r'l|litro', texto): return 'L'
    if re.search(r'ml', texto): return 'ml'
    return 'pz' # Por defecto 'pieza' si no encuentra nada

# Configuración de columnas (Mapeo exacto de Profeco)
nombres_cols = ["PRODUCTO", "PRESENTACIÓN", "MARCA", "CATEGORÍA", "CATÁLOGO", "PRECIO",
                "FECHAREGISTRO", "CADENACOMERCIAL", "GIRO", "NOMBRECOMERCIAL", "DIRECCIÓN",
                "ESTADO", "MUNICIPIO", "LATITUD", "LONGITUD"]
cols_usar = ["PRODUCTO", "PRESENTACIÓN", "MARCA", "CATEGORÍA", "PRECIO", "FECHAREGISTRO",
             "CADENACOMERCIAL", "NOMBRECOMERCIAL", "DIRECCIÓN", "ESTADO", "MUNICIPIO", "LATITUD", "LONGITUD"]

# --- B. Buscar archivos descargados en D: ---
# Usamos CARPETA_DATOS que definiste en la Celda 1
patron_busqueda = os.path.join(CARPETA_DATOS, "**", "*.csv")
archivos_csv = glob.glob(patron_busqueda, recursive=True)

print(f"📂 Buscando en: {CARPETA_DATOS}")
print(f"📄 Se encontraron {len(archivos_csv)} archivos CSV para procesar.")

total_registros = 0

# --- C. Conectar a Base de Datos ---
# Usamos las variables globales DB_USER, DB_PASS, etc. que definiste en la Celda 3
try:
    conn = psycopg2.connect(
        user=DB_USER, password=DB_PASS, host=DB_HOST, port=DB_PORT, dbname=DB_NAME
    )
    cur = conn.cursor()
except Exception as e:
    print("❌ Error conectando a BD. Asegúrate de haber corrido la Celda 3 primero.")
    print(f"Detalle: {e}")
    # Detenemos la ejecución si no hay conexión
    raise e

# --- D. Bucle Principal ---
for archivo in archivos_csv:
    try:
        # 1. Leer CSV
        # encoding='latin1' es crucial para archivos de gobierno en México
        df = pd.read_csv(archivo, names=nombres_cols, usecols=cols_usar, 
                         encoding='latin1', header=None, skiprows=1)
        
        # 2. Filtrar Geográficamente
        filtro = (df['ESTADO'].str.upper() == ESTADO_A_FILTRAR) & \
                 (df['MUNICIPIO'].str.upper() == CIUDAD_A_FILTRAR)
        df_clean = df[filtro].copy()

        if df_clean.empty:
            continue # Si no hay datos de Hermosillo, pasamos al siguiente archivo

        # 3. Transformación (Ingeniería de Características)
        # Extraemos cantidad y unidad
        df_clean['cantidad'] = df_clean['PRESENTACIÓN'].apply(extraer_cantidad)
        df_clean['unidad'] = df_clean['PRESENTACIÓN'].apply(extraer_unidad)

        # 4. Corrección lógica (Sanity Check)
        # Si dice "Litros" pero la cantidad es > 20 (ej. 950 L), asumimos que son ml
        mask_error_litros = (df_clean['unidad'] == 'L') & (df_clean['cantidad'] > 20)
        df_clean.loc[mask_error_litros, 'unidad'] = 'ml'

        # 5. Estandarización a Unidades Base (Kg y L)
        df_clean['cantidad_std'] = df_clean['cantidad']
        df_clean['unidad_std'] = df_clean['unidad']
        
        # Gramos -> Kg
        mask_gr = df_clean['unidad'] == 'gr'
        df_clean.loc[mask_gr, 'cantidad_std'] = df_clean.loc[mask_gr, 'cantidad'] / 1000
        df_clean.loc[mask_gr, 'unidad_std'] = 'kg'
        
        # Mililitros -> Litros
        mask_ml = df_clean['unidad'] == 'ml'
        df_clean.loc[mask_ml, 'cantidad_std'] = df_clean.loc[mask_ml, 'cantidad'] / 1000
        df_clean.loc[mask_ml, 'unidad_std'] = 'L'

        # 6. Calcular Precio Unitario
        # Evitamos división por cero reemplazando 0 con 1 (por seguridad)
        df_clean['cantidad_std'] = df_clean['cantidad_std'].replace(0, 1)
        df_clean['precio_unitario'] = df_clean['PRECIO'] / df_clean['cantidad_std']

        # 7. Preparar para SQL (Limpieza de Nulos)
        # Psycopg2 necesita None en lugar de NaN
        df_clean = df_clean.where(pd.notnull(df_clean), None)
        
        # Seleccionamos las columnas en el ORDEN EXACTO de la tabla SQL
        df_final = df_clean[[
            "PRODUCTO", "PRESENTACIÓN", "MARCA", "CATEGORÍA", 
            "PRECIO", "FECHAREGISTRO", "NOMBRECOMERCIAL", 
            "DIRECCIÓN", "ESTADO", "MUNICIPIO", "LATITUD", "LONGITUD",
            "cantidad_std", "unidad_std", "precio_unitario"
        ]]

        # Convertimos a lista de tuplas para la inserción rápida
        valores = [tuple(x) for x in df_final.to_numpy()]

        # 8. Insertar en Base de Datos
        insert_query = """
            INSERT INTO historico_precios_canasta 
            (producto_categoria, presentacion_original, marca, categoria_general, 
             precio_lista, fecha_registro, nombre_tienda, direccion, 
             estado, municipio, latitud, longitud,
             cantidad_estandarizada, unidad_estandarizada, precio_unitario) 
            VALUES %s
        """
        extras.execute_values(cur, insert_query, valores)
        conn.commit()
        
        registros_agregados = len(df_final)
        total_registros += registros_agregados
        print(f"  ✅ {os.path.basename(archivo)}: +{registros_agregados} registros guardados.")

    except Exception as e:
        print(f"  ⚠️ Error procesando archivo {os.path.basename(archivo)}: {e}")
        conn.rollback() # Deshacemos cambios de este archivo específico para no romper todo

# Cerramos conexión al finalizar
cur.close()
conn.close()
print(f"\n🎉 PROCESO FINALIZADO CON ÉXITO.")
print(f"📊 Se han cargado un total de {total_registros} registros limpios a PostgreSQL.")

🚀 Iniciando procesamiento inteligente y carga a SQL...
📂 Buscando en: D:\Documentos\ICD\datos_profeco_temp
📄 Se encontraron 191 archivos CSV para procesar.
  ✅ 062020.csv: +1770 registros guardados.
  ✅ 072020.csv: +2578 registros guardados.
  ✅ 082020.csv: +2791 registros guardados.
  ✅ 092020.csv: +3316 registros guardados.
  ✅ 102020.csv: +3358 registros guardados.
  ✅ 112020.csv: +1886 registros guardados.
  ✅ 122020.csv: +697 registros guardados.
  ✅ 162020.csv: +705 registros guardados.
  ✅ 172020.csv: +1766 registros guardados.
  ✅ 182020.csv: +1101 registros guardados.
  ✅ 192020.csv: +1214 registros guardados.
  ✅ 202020.csv: +1857 registros guardados.
  ✅ 212020.csv: +1869 registros guardados.
  ✅ 222020.csv: +2160 registros guardados.
  ✅ 152021.csv: +1159 registros guardados.
  ✅ 162021.csv: +1846 registros guardados.
  ✅ 172021.csv: +2202 registros guardados.
  ✅ 182021.csv: +2484 registros guardados.
  ✅ 192021.csv: +2398 registros guardados.
  ✅ 202021.csv: +2205 registr